### Export annotations from Labelbox to GCP storage

In [1]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload
from utils import gcp_utils
from utils.data_management import get_annotation_path
from utils.labelbox_utils import get_annotation_objects_from_data_row_export, get_geojson_fc_from_annotation_objects

In [4]:
# Load environment variables from .env.
# Alternatively, manually set environment variables.

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from google.cloud import storage
from labelbox import Client as LabelboxClient
from labelbox import Dataset

GCP_PROJECT_NAME = "gee-sand"
BUCKET_NAME = "sand_mining"
PROJECT_ID = 'cljoqdjei070j0729f20hc5sx'
LABELBOX_API_KEY = os.getenv('LABELBOX_API_KEY')

gcp_client = storage.Client(project="gee-sand")
labelbox_client = LabelboxClient(api_key=LABELBOX_API_KEY)

In [7]:
project = labelbox_client.get_project(PROJECT_ID)
export = project.export_v2()

export.wait_till_done()
if export.errors:
  print("Errors:")
  print(export.errors)
else:
  print("Finished without errors.")
  export_full_project = export.result

Finished without errors.


In [8]:
for data_row_export in export_full_project:
    global_key_data_row = data_row_export['data_row']['global_key']
    
    print(f"Uploading annoation to GCP for {global_key_data_row}")
    annotation_objects = get_annotation_objects_from_data_row_export(data_row_export)
    geojson_feature_collection = get_geojson_fc_from_annotation_objects(annotation_objects)
    annotation_destination_path = get_annotation_path(global_key_data_row)
    gcp_utils.upload_json(gcp_client, geojson_feature_collection, annotation_destination_path)
    

Uploading annoation to GCP for Ken_Banda_80-35_25-68_2022-03-29_rgb.tif
Uploading annoation to GCP for Ken_Banda_80-35_25-68_2022-03-09_rgb.tif
Uploading annoation to GCP for Ken_Banda_80-35_25-68_2022-04-08_rgb.tif
Uploading annoation to GCP for Kathajodi_Cuttack_85-85_20-44_2022-05-17_rgb.tif
Uploading annoation to GCP for Kathajodi_Cuttack_85-85_20-44_2022-03-03_rgb.tif
Uploading annoation to GCP for Kathajodi_Cuttack_85-85_20-44_2022-05-02_rgb.tif
